In this lab, you will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in
real-time.

TASK 1: Add a Launch Site Drop-down Input Component

In [3]:
pip install pandas dash


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install panel


Note: you may need to restart the kernel to use updated packages.


In [5]:
import panel as pn
import pandas as pd
import numpy as np

In [6]:
!pip install wget

In [8]:
#download csv file
import requests

url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv'
response = requests.get(url)

with open('spacex_launch_dash.csv', 'wb') as file:
    file.write(response.content)

# Now read the CSV file into a DataFrame
import pandas as pd
spacex_df = pd.read_csv('spacex_launch_dash.csv')
print(spacex_df.head(5))
      

   Unnamed: 0  Flight Number  Launch Site  class  Payload Mass (kg)  \
0           0              1  CCAFS LC-40      0                0.0   
1           1              2  CCAFS LC-40      0                0.0   
2           2              3  CCAFS LC-40      0              525.0   
3           3              4  CCAFS LC-40      0              500.0   
4           4              5  CCAFS LC-40      0              677.0   

  Booster Version Booster Version Category  
0  F9 v1.0  B0003                     v1.0  
1  F9 v1.0  B0004                     v1.0  
2  F9 v1.0  B0005                     v1.0  
3  F9 v1.0  B0006                     v1.0  
4  F9 v1.0  B0007                     v1.0  


TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown

In [12]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Load the CSV file
spacex_df = pd.read_csv('spacex_launch_dash.csv')

# Get the unique launch sites
launch_sites = spacex_df['Launch Site'].unique()
launch_sites = ['ALL'] + list(launch_sites)  # Include 'ALL' option

# Define the minimum and maximum payload values
min_payload = 0
max_payload = 10000

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("SpaceX Launch Data Dashboard"),
    
    # Dropdown for selecting a launch site
    dcc.Dropdown(
        id='site-dropdown',
        options=[{'label': site, 'value': site} for site in launch_sites],
        value='ALL',  # Default value
        placeholder="Select a Launch Site",
        searchable=True
    ),
    
    # Slider for selecting payload mass range
    dcc.RangeSlider(
        id='payload-slider',
        min=min_payload,
        max=max_payload,
        step=1000,
        marks={i: f'{i} Kg' for i in range(min_payload, max_payload + 1000, 1000)},
        value=[min_payload, max_payload]
    ),
    
    # Placeholder for the pie chart
    dcc.Graph(id='success-pie-chart'),

    # Placeholder for the scatter chart
    dcc.Graph(id='success-payload-scatter-chart')
])

# Callback to update the pie chart based on selected launch site and payload mass range
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value'),
    Input('payload-slider', 'value')
)
def update_pie_chart(selected_site, payload_range):
    # Filter data based on the payload range
    filtered_df = spacex_df[
        (spacex_df['Payload Mass (kg)'] >= payload_range[0]) & 
        (spacex_df['Payload Mass (kg)'] <= payload_range[1])
    ]
    
    if selected_site == 'ALL':
        # Show overall success rate for all launch sites within the payload range
        fig = px.pie(filtered_df, names='Launch Site', values='class',
                     title=f'Overall Success Rate by Launch Site (Payload Mass: {payload_range[0]} - {payload_range[1]} kg)')
    else:
        # Filter data for the selected site within the payload range
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]
        # Show success vs. failure for the selected site within the payload range
        fig = px.pie(filtered_df, names='class',
                     title=f'Success vs. Failure for {selected_site} (Payload Mass: {payload_range[0]} - {payload_range[1]} kg)')
    
    return fig

# Callback to update the scatter chart based on selected launch site and payload mass range
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    Input('site-dropdown', 'value'),
    Input('payload-slider', 'value')
)
def update_scatter_chart(selected_site, payload_range):
    # Filter data based on the payload range
    filtered_df = spacex_df[
        (spacex_df['Payload Mass (kg)'] >= payload_range[0]) & 
        (spacex_df['Payload Mass (kg)'] <= payload_range[1])
    ]
    
    if selected_site == 'ALL':
        # Show scatter plot for all sites within the payload range
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                         title=f'Success by Payload Mass for All Sites (Payload Mass: {payload_range[0]} - {payload_range[1]} kg)')
    else:
        # Filter data for the selected site within the payload range
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]
        # Show scatter plot for the selected site within the payload range
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                         title=f'Success by Payload Mass for {selected_site} (Payload Mass: {payload_range[0]} - {payload_range[1]} kg)')
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
# by opening it on the development server, http://127.0.0.1:8051/    